![NCIA](NCIA_Images\start.png)

# Building Neural Networks with PyTorch

### Setup

Este projeto requer Python 3.10 ou superior:

In [ ]:
!pip install -q pandas matplotlib seaborn scikit-learn

<div align="center" style="margin-top:20px; margin-bottom:20px;">
  
![pt](Aula_Imagens/pytorch.png)

</div>


Isso demora 9 minutos =/

In [ ]:
!pip install torch>=2.8.0

In [1]:
import sys
assert sys.version_info >= (3, 10)

Também requer Scikit-Learn ≥ 1.6.1:

In [2]:
from packaging.version import Version
import sklearn
assert Version(sklearn.__version__) >= Version("1.6.1")

E é claro que precisamos do PyTorch, especificamente do PyTorch ≥ 2.8.0:

In [3]:
import torch

assert Version(torch.__version__) >= Version("2.8.0")

Como fizemos nos capítulos anteriores, vamos definir os tamanhos de fonte padrão para deixar as figuras mais bonitas:

In [4]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

## Sobre o Pytorch

PyTorch é uma biblioteca open source de deep learning (originada do Torch/Lua) mantida pelo laboratório FAIR (Meta AI). Em Python, oferece **tensores**, **autodiferenciação (autograd)**, **aceleração por GPU/TPU** e um ecossistema de **camadas, perdas e otimizadores prontos**, tornando o desenvolvimento de redes neurais ágil e produtivo. Conceitualmente, lembra o NumPy, mas com suporte nativo a **cálculo em hardware acelerado** e **gradientes automáticos**.

"PôfêSô, por que Pt? eu gostava do TF!" Em 2016, o **TensorFlow** dominava o cenário por desempenho e escalabilidade, porém seu modelo de programação era mais **estático e complexo**. O PyTorch foi criado com uma proposta **mais “pythônica” e dinâmica**, usando **grafos computacionais dinâmicos (define-by-run)**, o que facilita a **depuração**, a **exploração interativa** e a **pesquisa**.
Além do design limpo e documentação robusta, a comunidade open source cresceu rapidamente; em 2022, a governança migrou para a **PyTorch Foundation** (Linux Foundation), consolidando o ecossistema. O resultado prático foi a **adoção massiva na academia** e, por consequência, **migração gradual da indústria**.

## O que você aprenderá neste capítulo

* **Fundamentos**: como trabalhar com **tensores** e **autograd** no PyTorch.
* **Primeiros passos de modelagem**: construir e treinar um **modelo de regressão linear** para entender o pipeline básico.
* **Evolução para redes profundas**: ampliar para **redes multicamadas (MLP)**:

  * **Regressão** com redes neurais.
  * **Classificação** com redes neurais.
* **Arquiteturas personalizadas**: criar modelos com **múltiplas entradas** ou **múltiplas saídas**.
* **Ajuste de hiperparâmetros**: usar **Optuna** para **tunar** modelos automaticamente.
* **Otimização e exportação**: técnicas para **otimizar desempenho** e **salvar/exportar** modelos para uso em produção.

> **Resumo da ideia central**: PyTorch equilibra **simplicidade**, **flexibilidade** e **alto desempenho** graças aos **grafos dinâmicos** e ao **ecossistema maduro**. Este capítulo guia você do **básico (tensores/autograd)** à **produção (tuning, otimização e exportação)** passando por exemplos práticos de **regressão** e **classificação**.


# 1. Fundamentos do PyTorch

O **tensor** é a estrutura de dados central do PyTorch: um **array multidimensional** com **forma** e **tipo de dado**, usado para computação numérica. Ele é semelhante a um array do **NumPy**, mas tem duas vantagens fundamentais: pode **residir em GPU** (ou outros aceleradores) e **suporta auto-diferenciação**. A partir deste ponto, **todos os modelos** trabalharão **recebendo e produzindo tensores**, de modo análogo a como modelos do Scikit-Learn usam arrays NumPy. O próximo passo é **criar e manipular tensores**.

## 1.1 PyTorch Tensors

Você pode criar um tensor PyTorch da mesma forma que criaria um array NumPy. Por exemplo, vamos criar um array 2 × 3:

In [5]:
import torch

X = torch.tensor([[1.0, 4.0, 7.0], [2.0, 3.0, 6.0]])
X

tensor([[1., 4., 7.],
        [2., 3., 6.]])

Assim como um array do NumPy, um **tensor** pode conter **floats**, **inteiros**, **booleanos** ou **números complexos** — **apenas um tipo por tensor**. Se você o inicializa com valores de tipos mistos, o PyTorch escolhe o **mais geral** segundo a hierarquia: **complexo > float > inteiro > bool**. Também é possível **definir o tipo explicitamente** na criação (por exemplo, `dtype=torch.float16` para floats de 16 bits). **Tensores de strings ou objetos não são suportados**.

> Você pode **inspecionar a forma (shape) e o tipo (dtype)** de um tensor diretamente:


In [6]:
X.shape

torch.Size([2, 3])

In [7]:
X.dtype

torch.float32

A indexação funciona exatamente como para matrizes NumPy:

In [8]:
X[0, 1]

tensor(4.)

In [9]:
X[:, 1]

tensor([4., 3.])

Você pode aplicar **diversas operações numéricas** diretamente em tensores, com uma API **muito semelhante à do NumPy**: `torch.abs()`, `torch.cos()`, `torch.exp()`, `torch.max()`, `torch.mean()`, `torch.sqrt()`, entre outras.
Quase todas também existem como **métodos do próprio tensor**, permitindo escrever `X.exp()` em vez de `torch.exp(X)`. O próximo trecho demonstra algumas dessas operações na prática.

In [10]:
10 * (X + 1.0)  # item-wise addition and multiplication

tensor([[20., 50., 80.],
        [30., 40., 70.]])

In [11]:
X.exp()

tensor([[   2.7183,   54.5981, 1096.6332],
        [   7.3891,   20.0855,  403.4288]])

In [12]:
X.mean()

tensor(3.8333)

In [13]:
X.max(dim=0)

torch.return_types.max(
values=tensor([2., 4., 7.]),
indices=tensor([1, 0, 0]))

In [14]:
X @ X.T

tensor([[66., 56.],
        [56., 49.]])

Você também pode converter um tensor em uma matriz NumPy usando o método numpy() e criar um tensor a partir de uma matriz NumPy:

In [15]:
import numpy as np

X.numpy()

array([[1., 4., 7.],
       [2., 3., 6.]], dtype=float32)

In [16]:
torch.tensor(np.array([[1., 4., 7.], [2., 3., 6.]]))

tensor([[1., 4., 7.],
        [2., 3., 6.]], dtype=torch.float64)

No PyTorch, o **padrão de floats é 32 bits (float32)**, enquanto no NumPy é **64 bits (float64)**. Em deep learning, **float32 costuma ser preferível**: consome **metade da RAM**, **acelera os cálculos** e a rede **não precisa** da precisão extra de 64 bits.
Ao converter um array NumPy com `torch.tensor()`, **indique** `dtype=torch.float32`. Como alternativa, `torch.FloatTensor()` **já converte** automaticamente para **32 bits**.

In [17]:
torch.tensor(np.array([[1., 4., 7.], [2., 3., 6.]]), dtype=torch.float32)

tensor([[1., 4., 7.],
        [2., 3., 6.]])

In [18]:
torch.FloatTensor(np.array([[1., 4., 7.], [2., 3., 6]]))

tensor([[1., 4., 7.],
        [2., 3., 6.]])

In [19]:
# extra code: demonstrate torch.from_numpy()
X2_np = np.array([[1., 4., 7.], [2., 3., 6]])
X2 = torch.from_numpy(X2_np)  # X2_np and X2 share the same data in memory
X2_np[0, 1] = 88
X2

tensor([[ 1., 88.,  7.],
        [ 2.,  3.,  6.]], dtype=torch.float64)

Você também pode modificar um tensor no local usando indexação e fatiamento, como com uma matriz NumPy:

In [20]:
X[:, 1] = -99
X

tensor([[  1., -99.,   7.],
        [  2., -99.,   6.]])

A API do PyTorch oferece diversas operações **in-place** (terminadas com `_`), como `abs_()`, `sqrt_()` e `zero_()`, que **modificam o próprio tensor**. Elas podem **economizar memória** e **aumentar a velocidade** em alguns casos.
Exemplo: `relu_()` aplica a **ReLU** diretamente, **substituindo valores negativos por 0** no mesmo tensor.


In [21]:
X.relu_()
X

tensor([[1., 0., 7.],
        [2., 0., 6.]])

Os tensores do PyTorch realmente se assemelham a matrizes NumPy. Na verdade, eles têm mais de 200 funções comuns!

In [22]:
# extra code: list functions that appear both in NumPy and PyTorch
functions = lambda mod: set(f for f in dir(mod) if callable(getattr(mod, f)))
", ".join(sorted(functions(torch) & functions(np)))

'__getattr__, abs, absolute, acos, acosh, add, all, allclose, amax, amin, angle, any, arange, arccos, arccosh, arcsin, arcsinh, arctan, arctan2, arctanh, argmax, argmin, argsort, argwhere, asarray, asin, asinh, atan, atan2, atanh, atleast_1d, atleast_2d, atleast_3d, bincount, bitwise_and, bitwise_left_shift, bitwise_not, bitwise_or, bitwise_right_shift, bitwise_xor, broadcast_shapes, broadcast_to, can_cast, ceil, clip, column_stack, concat, concatenate, conj, copysign, corrcoef, cos, cosh, count_nonzero, cov, cross, cumprod, cumsum, deg2rad, diag, diagflat, diagonal, diff, divide, dot, dsplit, dstack, dtype, einsum, empty, empty_like, equal, exp, exp2, expm1, eye, finfo, fix, flip, fliplr, flipud, float_power, floor, floor_divide, fmax, fmin, fmod, frexp, from_dlpack, frombuffer, full, full_like, gcd, gradient, greater, greater_equal, heaviside, histogram, histogramdd, hsplit, hstack, hypot, i0, iinfo, imag, inner, isclose, isfinite, isin, isinf, isnan, isneginf, isposinf, isreal, kron

## 1.2 Hardware Acceleration

**Copiar tensores para a GPU** é simples no PyTorch, desde que sua máquina tenha GPU compatível e as bibliotecas necessárias estejam instaladas. No **Google Colab**, basta usar um **runtime com GPU** (Menu *Runtime* → *Change runtime type* → selecionar uma GPU, como **Nvidia T4**). Esse ambiente já vem com o PyTorch compilado com suporte a GPU, drivers e bibliotecas requeridas (por exemplo, **CUDA** e **cuDNN**).
Se preferir rodar localmente, instale **drivers** e **bibliotecas** apropriadas seguindo as instruções: [https://homl.info/install-p](https://homl.info/install-p).

O PyTorch tem excelente suporte a **GPUs Nvidia** e também a outros aceleradores:

* **Apple MPS**: aceleração em **Apple Silicon** (M1, M2, posteriores) e alguns **Intel Macs** compatíveis.
* **AMD**: **Instinct** e **Radeon** via **ROCm** (Linux) ou **DirectML** (Windows).
* **Intel**: **GPUs e CPUs** (Linux/Windows) via **oneAPI**.
* **Google TPUs**: integração via **`torch_xla`**.


In [23]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device

'cpu'

Em um Colab GPU Runtime, o dispositivo será igual a "cuda". Agora, vamos criar um tensor nessa GPU. Para isso, uma opção é criar o tensor na CPU e copiá-lo para a GPU usando o método **to()**:

In [24]:
M = torch.tensor([[1., 2., 3.], [4., 5., 6.]])
M = M.to(device)
M.device

device(type='cpu')

Alternativamente, podemos criar o tensor diretamente na GPU usando o argumento do dispositivo:

In [25]:
M = torch.tensor([[1., 2., 3.], [4., 5., 6.]], device=device)

Quando o tensor estiver na GPU, podemos executar operações nele normalmente, e todas elas ocorrerão na GPU:

In [26]:
R = M @ M.T  # run some operations on the GPU
R

tensor([[14., 32.],
        [32., 77.]])

Quando um tensor é processado na **GPU**, o **resultado também permanece na GPU**. Isso permite **encadear várias operações** sem **copiar dados entre CPU e GPU**, evitando um gargalo comum de desempenho.

**Quanto a GPU acelera?** Depende do **modelo da GPU** (as mais caras podem ser **dezenas de vezes** mais rápidas) e do **throughput de dados**:

* **Modelos compute-heavy** (ex.: redes muito profundas): o **poder de cálculo** da GPU e a **quantidade de RAM** tendem a ser os fatores críticos.
* **Modelos rasos / datasets grandes**: o **envio contínuo de dados** para a GPU pode virar o **gargalo** principal.

O próximo trecho realiza um **teste comparando** a **multiplicação de matrizes** na **CPU vs GPU** para ilustrar essas diferenças de **tempo de execução** e **banda de dados**.


In [27]:
M = torch.rand((1000, 1000))  # on the CPU
M @ M.T  # warmup
%timeit M @ M.T

M = M.to(device)
M @ M.T  # warmup
%timeit M @ M.T

3.34 ms ± 187 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.58 ms ± 149 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


No teste, a **GPU (Nvidia T4 no Colab)** proporcionou um **ganho de ~26×** na multiplicação de matrizes. Com GPUs mais potentes, o **speedup** tende a ser ainda maior. Porém, **em matrizes pequenas** (ex.: `100 × 100`), o ganho cai para algo como **~2×**.
Isso ocorre porque **GPUs paralelizam tarefas grandes** (quebrando-as em muitas subtarefas para milhares de núcleos). **Tarefas pequenas** não geram paralelismo suficiente, reduzindo o benefício — e, em cenários com **muitas tarefas minúsculas**, **a CPU pode ser até mais rápida** devido ao overhead de orquestração na GPU.

Com a base de **tensores** e **execução em CPU/GPU** estabelecida, o próximo passo é explorar o **autograd** do PyTorch (auto-diferenciação).


![NCIA2](NCIA_Images\end.png)